In [2]:
import pymysql.cursors
#import jsonify
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle
from sklearn.ensemble import RandomForestRegressor

In [4]:
conn = pymysql.connect(
    host="softwaredb.ce0otalnccc9.eu-west-1.rds.amazonaws.com",user="soft",password="password",database="dublinbikes",
    charset="utf8mb4",
    cursorclass=pymysql.cursors.DictCursor
)


In [5]:
availability_query = "SELECT number, time, available_bikes, available_bike_stands FROM availability"
#weather_query = "SELECT number, time, temperature, wind_speed, wind_direction, weather_code FROM weather"
weather_query = "SELECT * FROM weather"
# Load the query results into pandas dataframes
availability_df = pd.read_sql_query(availability_query, conn)
weather_df = pd.read_sql_query(weather_query, conn)

conn.close()

C:\Users\tfiro\AppData\Local\Temp\ipykernel_17600\2178617842.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  availability_df = pd.read_sql_query(availability_query, conn)
C:\Users\tfiro\AppData\Local\Temp\ipykernel_17600\2178617842.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  weather_df = pd.read_sql_query(weather_query, conn)


In [6]:
availability_df['time'] = availability_df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
weather_df['time'] = weather_df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
merged_df = pd.merge(availability_df, weather_df, on=["number", "time"])

In [7]:
merged_df.to_csv('merged_data.csv', index=False)

In [5]:
merged_df = pd.read_csv("merged_data.csv")

In [6]:
merged_df.tail()

,number,time,available_bikes,available_bike_stands,temperature,wind_speed,wind_direction,weather_code,precipitation_sum,rain_sum,precipitation_probability
1313870,507,2023-03-10 16:40:02,1,0,6,3,76,1,8.1,3.1,100
1313871,507,2023-03-10 16:45:03,1,0,6,3,76,1,8.1,3.1,100
1313872,507,2023-03-10 16:50:03,1,0,6,3,76,1,8.1,3.1,100
1313873,507,2023-03-10 16:55:03,1,0,6,3,76,1,8.1,3.1,100
1313874,507,2023-03-10 17:00:07,1,0,5,8,90,1,8.1,3.1,100


In [48]:
merged_df.isnull().sum()

number                       0
time                         0
available_bikes              0
available_bike_stands        0
temperature                  0
wind_speed                   0
wind_direction               0
weather_code                 0
precipitation_sum            0
rain_sum                     0
precipitation_probability    0
dtype: int64

In [49]:
merged_df.describe().T

,count,mean,std,min,25%,50%,75%,max
number,1313647.0,58.856302,35.967582,1.0,29.0,58.0,89.0,507.0
available_bikes,1313647.0,11.505513,9.383810,0.0,4.0,10.0,17.0,40.0
available_bike_stands,1313647.0,20.054286,10.870357,0.0,12.0,20.0,29.0,40.0
temperature,1313647.0,9.041181,3.253094,1.0,7.0,9.0,11.0,19.0
wind_speed,1313647.0,16.903415,8.588150,1.0,11.0,16.0,22.0,44.0
wind_direction,1313647.0,199.609700,75.920632,3.0,146.0,214.0,262.0,360.0
weather_code,1313647.0,16.149440,23.068453,0.0,2.0,3.0,51.0,73.0
precipitation_sum,1313647.0,3.347413,4.983085,0.0,0.2,1.9,3.5,30.2
rain_sum,1313647.0,2.559542,3.872297,0.0,0.0,0.6,3.4,20.2
precipitation_probability,1313647.0,74.436469,31.617998,0.0,58.0,87.0,100.0,100.0


In [7]:
print(merged_df.dtypes)


number                         int64
time                          object
available_bikes                int64
available_bike_stands          int64
temperature                    int64
wind_speed                     int64
wind_direction                 int64
weather_code                   int64
precipitation_sum            float64
rain_sum                     float64
precipitation_probability      int64
dtype: object


In [8]:
merged_df['time'] = merged_df['time'].astype('datetime64')
merged_df['number'] = merged_df['number'].astype('category')
merged_df['weather_code'] = merged_df['weather_code'].astype('category')

In [9]:
merged_df['hour'] = merged_df['time'].dt.hour
merged_df['minute'] = merged_df['time'].dt.minute
merged_df['month'] = merged_df['time'].dt.month
merged_df['day'] = merged_df['time'].dt.day


In [10]:
merged_df['hour'] = merged_df['hour'].astype('category')
merged_df['minute'] = merged_df['minute'].astype('category')
merged_df['month'] = merged_df['month'].astype('category')
merged_df['day'] = merged_df['day'].astype('category')

In [11]:
station_dfs = {}
for station in merged_df['number'].unique():
    station_dfs[station] = merged_df[merged_df['number'] == station]


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
import numpy as np

# Assuming station_dfs is a dictionary containing data for each station

for station, station_df in station_dfs.items():
    X = station_df[['hour', 'day','minute','month','temperature', 'wind_speed', 'wind_direction', 'weather_code','precipitation_sum','rain_sum','precipitation_probability']]
    y = station_df['available_bikes']

    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    model = KNeighborsRegressor(n_neighbors=2)

    model.fit(X_train, y_train)
    print(f"Station {station} R-squared: {model.score(X_test, y_test)}")
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print("Mean Absolute Error: ", mae)
    print("Root Mean Squared Error: ", rmse)
    station_num = 1
    time_str = '2023-03-07 19:35:03'
    weather_code = 801
    temperature = 10
    wind_speed = 6
    wind_direction = 240
    precipitation_sum = 0
    rain_sum = 1
    precipitation_probability = 0
    time = pd.to_datetime(time_str)
    hour = time.hour
    day=time.day
    minute=time.minute
    month=time.month
    prediction_input = [[hour,day,minute,month, temperature, wind_speed, wind_direction, weather_code,precipitation_sum,rain_sum,precipitation_probability]]
    prediction = model.predict(prediction_input)
    
    print(f"Predicted number of available bikes for Station {station} at {time_str}: {prediction[0]:.2f}")


Station 1 R-squared: 0.9226939307715553
Mean Absolute Error:  1.272289156626506
Root Mean Squared Error:  2.4384682418701487
Predicted number of available bikes for Station 1 at 2023-03-07 19:35:03: 11.00
Station 2 R-squared: 0.8323448781174214
Mean Absolute Error:  0.8707401032702238
Root Mean Squared Error:  1.737879095953301
Predicted number of available bikes for Station 2 at 2023-03-07 19:35:03: 7.00


C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


Station 3 R-squared: 0.9045718756868758
Mean Absolute Error:  0.8795180722891566
Root Mean Squared Error:  1.612771745237844
Predicted number of available bikes for Station 3 at 2023-03-07 19:35:03: 10.00
Station 4 R-squared: 0.8684331928232801
Mean Absolute Error:  0.9860585197934596
Root Mean Squared Error:  2.1373451771289105
Predicted number of available bikes for Station 4 at 2023-03-07 19:35:03: 6.00


C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


Station 5 R-squared: 0.8787806260535712
Mean Absolute Error:  2.338037865748709
Root Mean Squared Error:  5.019171506950027
Predicted number of available bikes for Station 5 at 2023-03-07 19:35:03: 0.00
Station 6 R-squared: 0.9170628153869201
Mean Absolute Error:  0.8406196213425129
Root Mean Squared Error:  1.819099283463689
Predicted number of available bikes for Station 6 at 2023-03-07 19:35:03: 2.00
Station 7 R-squared: 0.8816418323416391
Mean Absolute Error:  1.0139414802065405
Root Mean Squared Error:  2.325437422994138
Predicted number of available bikes for Station 7 at 2023-03-07 19:35:03: 5.00
Station 8 R-squared: 0.8416061623280693
Mean Absolute Error:  1.5752151462994837
Root Mean Squared Error:  3.089290813124442
Predicted number of available bikes for Station 8 at 2023-03-07 19:35:03: 11.50
Station 9 R-squared: 0.8376419313354377
Mean Absolute Error:  1.687263339070568
Root Mean Squared Error:  3.0996779578434905
Predicted number of available bikes for Station 9 at 2023-0

...

C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


Mean Absolute Error:  1.767986230636833
Root Mean Squared Error:  3.860921206725655
Predicted number of available bikes for Station 114 at 2023-03-07 19:35:03: 30.00
Station 115 R-squared: 0.922277970625501
Mean Absolute Error:  1.2669535283993116
Root Mean Squared Error:  2.6145857262276673
Predicted number of available bikes for Station 115 at 2023-03-07 19:35:03: 2.00
Station 116 R-squared: 0.8541075293196135
Mean Absolute Error:  0.3387263339070568
Root Mean Squared Error:  0.7505591719298261
Predicted number of available bikes for Station 116 at 2023-03-07 19:35:03: 0.00


C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


Station 117 R-squared: 0.9306100978872315
Mean Absolute Error:  0.9249569707401033
Root Mean Squared Error:  2.6183846237452872
Predicted number of available bikes for Station 117 at 2023-03-07 19:35:03: 35.00
Station 507 R-squared: 1.0
Mean Absolute Error:  0.0
Root Mean Squared Error:  0.0
Predicted number of available bikes for Station 507 at 2023-03-07 19:35:03: 1.00


C:\Users\tfiro\anaconda3\envs\comp30830software\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd

# Assuming station_dfs is a dictionary containing data for each station

for station, station_df in station_dfs.items():
    X = station_df[['hour', 'day','minute','month','temperature', 'wind_speed', 'wind_direction', 'weather_code','precipitation_sum','rain_sum','precipitation_probability']]
    y = station_df['available_bikes']

    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    model = KNeighborsRegressor(n_neighbors=2)

    model.fit(X_train, y_train)
    print(f"Station {station} R-squared: {model.score(X_test, y_test)}")
    filename = f"model_dep{station}.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(model, file, pickle.HIGHEST_PROTOCOL)

Station 1 R-squared: 0.9156129820453823
Station 2 R-squared: 0.8139732090910414
Station 3 R-squared: 0.8974281675072003
Station 4 R-squared: 0.8758418241152907
Station 5 R-squared: 0.8937113982365168
Station 6 R-squared: 0.924982858210629
Station 7 R-squared: 0.8713384039490079
Station 8 R-squared: 0.8578761722346875
Station 9 R-squared: 0.8513977414410756
Station 10 R-squared: 0.8375305136927158
Station 11 R-squared: 0.9237049768285951
Station 12 R-squared: 0.9131758533977925
Station 13 R-squared: 0.9346249442205113
Station 14 R-squared: 0.842278463269348
Station 15 R-squared: 0.8224218527759979
Station 16 R-squared: 0.9046819197565575
Station 17 R-squared: 0.8939986320613262
Station 18 R-squared: 0.8875232771533224
Station 19 R-squared: 0.857252499501178
Station 20 R-squared: 0.9044153623950191
Station 21 R-squared: 0.8697127782309535
Station 22 R-squared: 0.8957632394400598
Station 23 R-squared: 0.8814297832401556
Station 24 R-squared: 0.8299521034201908
Station 25 R-squared: 0.9223

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd

# Assuming station_dfs is a dictionary containing data for each station

for station, station_df in station_dfs.items():
    X = station_df[['hour', 'day','minute','month','temperature', 'wind_speed', 'wind_direction', 'weather_code','precipitation_sum','rain_sum','precipitation_probability']]
    y = station_df['available_bike_stands']

    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    model = KNeighborsRegressor(n_neighbors=2)

    model.fit(X_train, y_train)
    print(f"Station {station} R-squared: {model.score(X_test, y_test)}")
    filename = f"model_arr{station}.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(model, file, pickle.HIGHEST_PROTOCOL)

Station 1 R-squared: 0.9054055329909676
Station 2 R-squared: 0.8382245406769573
Station 3 R-squared: 0.9009472636894702
Station 4 R-squared: 0.8827480394387843
Station 5 R-squared: 0.8865130523316969
Station 6 R-squared: 0.9053629545632578
Station 7 R-squared: 0.8858747081539521
Station 8 R-squared: 0.8452494451747148
Station 9 R-squared: 0.8418498692112817
Station 10 R-squared: 0.8402041835219073
Station 11 R-squared: 0.9492257932235814
Station 12 R-squared: 0.9083111306097099
Station 13 R-squared: 0.919698129528037
Station 14 R-squared: 0.8645694048133156
Station 15 R-squared: 0.84783151618211
Station 16 R-squared: 0.8893114422701045
Station 17 R-squared: 0.9140432108656705
Station 18 R-squared: 0.8902292562221273
Station 19 R-squared: 0.8671320879199823
Station 20 R-squared: 0.9043826287197976
Station 21 R-squared: 0.8791191142690854
Station 22 R-squared: 0.9005930848312711
Station 23 R-squared: 0.8591887713407063
Station 24 R-squared: 0.8553310560253647
Station 25 R-squared: 0.9123